In [ ]:
#
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#I will need these functions
fd = tf.contrib.distributions

#switch to convolutional
def make_encoder(data, code_size):
    x = tf.layers.flatten(data)
    x = tf.layers.dense(x, 200, tf.nn.relu)
    x = tf.layers.dense(x, 200, tf.nn.relu) # encode data 
    loc = tf.layers.dense(x, code_size) # generate latent layer mean 
    #code_size defines the number of latent parameters
    scale = tf.layers.dense(x, code_size, tf.nn.softplus) # latent layer sigma
    return tfd.MultivariateNormalDiag(loc, scale)

# set prior using independent-MVN 
def make_prior(code_size):
    loc = tf.zeros(code_size)
    scale = tf.ones(code_size)
    return tfd.MultivariateNormalDiag(loc, scale)


# generate data from latent "code" space
def make_decoder(code, data_shape):
    x = code
    x = tf.layers.dense(x, 200, tf.nn.relu)
    x = tf.layers.dense(x, 200, tf.nn.relu)
    logit = tf.layers.dense(x, np.prod(data_shape)) 
    logit = tf.reshape(logit, [-1] + data_shape) # generate an image the size of the data
    return tfd.Independent(tfd.Bernoulli(logit), 2) # change to binary (training is MINST data set)


def plot_codes(ax, codes, labels):
    ax.scatter(codes[:, 0], codes[:, 1], s=2, c=labels, alpha=0.1)
    ax.set_aspect('equal')
    ax.set_xlim(codes.min() - .1, codes.max() + .1)
    ax.set_ylim(codes.min() - .1, codes.max() + .1)
    ax.tick_params(
      axis='both', which='both', left='off', bottom='off',
      labelleft='off', labelbottom='off')
def plot_samples(ax, samples):
      for index, sample in enumerate(samples):
        ax[index].imshow(sample, cmap='gray')
        ax[index].axis('off')